In [ ]:
# 내 구글 드라이버에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import time

In [ ]:
# Object tracker mode 설정

# csrt
# tracker = cv2.TrackerCSRT_create()
# kcf
tracker = cv2.TrackerKCF_create()
# boosting
# tracker = cv2.TrackerBoosting_create()
# mil
# tracker = cv2.TrackerMIL_create()
# tld
# tracker = cv2.TrackerTLD_create()
# medianflow
# tracker = cv2.TrackerMedianFlow_create()
# mosse
# tracker = cv2.TrackerMOSSE_create()

In [ ]:
face_cascade_name = 'gdrive/My Drive/CV/Face Detection/data/haarcascades/haarcascade_frontalface_alt.xml'  # 얼굴을 인식하는 haarcascade feature 파일
face_cascade = cv2.CascadeClassifier()   # face_cascade 인스턴스를 생성한다.
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('### Error loading face cascade ###')
    exit(0)

In [ ]:
file_name = 'Lisa.mp4'             # 원본 동영상
output_name = 'face_tracking_video.mp4'  # Tracking 된 output 동영상
frame_count = 0            # frame count 초기화
detected = False           # detected mode 초기화
frame_mode = 'Tracking'    # frame mode 초기화
elapsed_time = 0           # 총 경과시간 초기화

In [ ]:
# 원본 동영상을 읽어옴
cap = cv2.VideoCapture(file_name)

# 원본 동영상이 오픈되는지 확인
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

In [ ]:
writer = None    # writer 초기화

trackers = cv2.MultiTracker_create()  # trackers 객체를 생성합니다.

while True:
  # 원본 동영상에서 frame 을 읽는다
  ret, frame = cap.read()

  # 원본 동영상에서 더 이상 frame 을 읽지 못했다면 Exit
  if frame is None:
     # close the video file pointers
     cap.release()
     # close the writer point
     writer.release()
     print('--(!) No captured frame -- Break!')
     print("elapsed time {:.3f} seconds".format(elapsed_time))
     break

  start_time = time.time()
  frame_count += 1    # frame count 누적

  if detected:    
      frame_mode = 'Tracking'  # Tracking mode
      # trackers 객체가 tracking 하여 변경된 boxing(얼굴) 영역 위치 좌표를 업데이트함
      (success, boxes) = trackers.update(frame)
      for box in boxes: 
          (x, y, w, h) = [int(v) for v in box]
          cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # 변경된 위치 좌표에 boxing
  else:
      frame_mode = 'Detection'   # Detection mode
      frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      frame_gray = cv2.equalizeHist(frame_gray)            
      faces = face_cascade.detectMultiScale(frame_gray)

      largest_box = 0   # largest_box 초기화

      for (_x,_y,_w,_h) in faces:  # detect된 box 중에서 largest box 를 찾는다
         SZ = _w * _h
         if SZ > largest_box:
            largest_box = SZ
            x, y, w, h = int(_x), int(_y), int(_w), int(_h) 

      cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 4)  # Detect 된 얼굴에 boxing

      # trackers 객체가 설정된 tracker mode 로 boxing(얼굴)된 영역을 tracking 하도록 설정
      trackers.add(tracker, frame, (x, y, w, h))
      detected = True    # detected mode 변경

  # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
  if writer is None and output_name is not None:
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(output_name, fourcc, 30, (frame.shape[1], frame.shape[0]), True)

  # disk 에 frame 을 write 합니다.
  if writer is not None:
    writer.write(frame)

  frame_time = time.time() - start_time
  elapsed_time += frame_time   # 총 경과시간 누적
  print("[{}] Frame {} time {}".format(frame_mode, frame_count, frame_time))

[Detection] Frame 1 time 0.6662747859954834
[Tracking] Frame 2 time 0.023573875427246094
[Tracking] Frame 3 time 0.03014206886291504
[Tracking] Frame 4 time 0.029692649841308594
[Tracking] Frame 5 time 0.031400442123413086
[Tracking] Frame 6 time 0.029837369918823242
[Tracking] Frame 7 time 0.0211331844329834
[Tracking] Frame 8 time 0.028563976287841797
[Tracking] Frame 9 time 0.02604508399963379
[Tracking] Frame 10 time 0.02481842041015625
[Tracking] Frame 11 time 0.020418167114257812
[Tracking] Frame 12 time 0.03511691093444824
[Tracking] Frame 13 time 0.028630733489990234
[Tracking] Frame 14 time 0.0283811092376709
[Tracking] Frame 15 time 0.03171038627624512
[Tracking] Frame 16 time 0.02895212173461914
[Tracking] Frame 17 time 0.03050827980041504
[Tracking] Frame 18 time 0.03263592720031738
[Tracking] Frame 19 time 0.03150606155395508
[Tracking] Frame 20 time 0.029348373413085938
[Tracking] Frame 21 time 0.0289614200592041
[Tracking] Frame 22 time 0.029557228088378906
[Tracking] Fr